In [ ]:
import sys
import numpy as np
import pickle as pkl
from IPython.display import display
from maraboupy import Marabou, MarabouCore
from utils import TOTUtils

In [1]:
import pandas as pd
samples_path = '../logs/samples.csv'
samples = pd.read_csv(samples_path, index_col=0)
samples = [(s[0:25].tolist(), s[25:].tolist()) for s in samples.values][0:2]

In [2]:
from tot_net import TOTNet
from maraboupy import Marabou, MarabouUtils
from sensitivity import test_sensitivity

nnet_path = '../network/models/v3.2.0/model.nnet'
sym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, save_results=True, save_samples=True, verbose=2, outdir='../logs/sensitivity/symmetric')
asym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, asym=True, save_results=True, save_samples=True, verbose=2, outdir='../logs/sensitivity/asymmetric')

print('SYM RESULTS:', sym_results)
print('ASYM RESULTS:', asym_results)

INFO:sensitivity:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:sensitivity:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.01 and 0.09
INFO:sensitivity:searching 9 coarse epsilons b/t 0.1 and 0.9
INFO:sensitivity:searching 9 coarse epsilons b/t 1.0 and 9.0
INFO:sensitivity:x0_s0 interm epsilon bounds: (4.0, 5.0)
INFO:sensitivity:searching 100000 epsilons b/t 4.0 and 4.99999
INFO:sensitivity:x0_s0 epsilon: 4.1331
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-06 and 9e-06
INFO:sensitivity:searching 9 coarse epsilons b/t 1e-05 and 9e-05
INFO:sensitivity:searching 9 coarse epsilons b/t 0.0001 and 0.0009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.001 and 0.009
INFO:sensitivity:searching 9 coarse epsilons b/t 0.01 and 0.09
INFO:sensitivity:searching 9 coarse epsilons b/t 0.1 and 0.9
INFO:sen

In [8]:
from tot_net import TOTNet

def check_input_epsilons(net, sample, x, le, ue, asym=False):
    inputs, outputs = sample
    y = outputs.index(max(outputs))
    lbs = inputs[0:x] + [inputs[x]+le] + inputs[x+1:]
    ubs = inputs[0:x] + [inputs[x]+ue] + inputs[x+1:]
    result = None
    if asym:
        result = (net.find_counterexample(lbs, inputs, y),
                  net.find_counterexample(inputs, ubs, y))
    else:
        result = net.find_counterexample(lbs, ubs, y)
    return result

sym_tests = {}
asym_tests = {}
net = TOTNet(nnet_path)
nnet_path = '../network/models/v3.2.0/model.nnet'
for x in range(net.get_num_inputs()):
    xid = f'x{x}'
    sym_tests[xid] = []
    asym_tests[xid] = []
    for sample in samples:
        sym_le, sym_ue = sym_results[xid][0]
        sym_cexs = check_input_epsilons(net, sample, x, sym_le, sym_ue)
        sym_tests[xid].append(sym_cexs)

        asym_le, asym_ue = asym_results[xid][0]
        asym_cexs = check_input_epsilons(net, sample, x, asym_le, asym_ue, asym=True)
        asym_tests[xid].append(asym_cexs)

print('SYM TESTS:', sym_tests)
print('ASYM TESTS:', asym_tests)

IndexError: list index out of range

In [ ]:
# model_path = '../artifacts/models/model_v3.2.0/model.h5'
# data_csv = '../data/v3.2.0/test.csv'
# all_samples = TOTUtils.load_samples(data_csv, frac=0.0001)
# samples = TOTUtils.filter_samples(all_samples, model_path)
# print(f'{len(samples)} correct samples')
from tot_net import TOTNet
nnet_path = '../network/models/v3.2.0/model.nnet'
net = TOTNet(nnet_path)
sample = samples[0]
inputs, outputs = sample
net.evaluate([inputs])

In [ ]:
# from sensitivity import find_feature_sensitivity_boundaries
# import pandas as pd
# n_inputs=25
# samples = pd.read_csv('../logs/sensitivity/samples.csv', index_col=0, header=0)
# samples = [(s[0:n_inputs].tolist(), s[n_inputs:].tolist()) for s in samples.values]
# find_feature_sensitivity_boundaries('../artifacts/models/model_v3.2.0/model.h5', 0, samples, d_min=0.00001, d_max=100, verbose=1)

In [ ]:
# from sensitivity import find_sensitivity_boundaries

# results = find_sensitivity_boundaries(model_path, samples[:10], d_min=0.00001, d_max=100, multithread=True, save_results=True, save_samples=True)

# for x,result in results.items():
#     dists,_ = result
#     print(x, dists)

In [ ]:
# from tensorflow.keras.models import load_model

# model = load_model('../artifacts/models/model_v3.2.0/model.h5')
# inputs,outputs = samples[6]
# d = 49.69331
# x = 0
# pred = model.predict([inputs[0:x] + [inputs[x]+d] + inputs[x+1:]])

# # assert that prediction was correct
# assert(outputs.index(max(outputs)) == list(pred[0]).index(max(pred[0])))

In [ ]:
# from maraboupy import Marabou, MarabouCore

# pb_path = '../artifacts/models/model-v3.1.4/model.pb'
# net = Marabou.read_tf(pb_path)

# sample = samples[0]
# inputs, outputs = sample
# expected_cat = outputs.index(max(outputs))
# other_cats = [i for i in range(len(outputs)) if i != expected_cat]

# for x,v in enumerate(inputs):
#     net.setLowerBound(net.inputVars[0][x], v)
#     net.setUpperBound(net.inputVars[0][x], v)
# for c in other_cats:
#     print('OC:', c)
#     eq1 = MarabouCore.Equation(MarabouCore.Equation.LE)
#     eq1.addAddend(1, net.outputVars[0][c])
#     eq1.addAddend(-1, net.outputVars[0][0])
#     eq1.setScalar(0)
# pred = net.evaluate([input_sample], useMarabou=False)
# print(pred)

In [ ]:
from maraboupy import Marabou, MarabouCore

nnet_path = '../artifacts/models/v3.2.0/model.nnet'
net = Marabou.read_nnet(nnet_path)
samples = pd.read_csv('../logs/sensitivity/samples.csv', index_col=0, header=0)
samples = [(s[0:n_inputs].tolist(), s[n_inputs:].tolist()) for s in samples.values]
# samples = samples[0:10]
sample = samples[0]
inputs, outputs = sample
expected_cat = outputs.index(max(outputs))
other_cats = [i for i in range(len(outputs)) if i != expected_cat]
for x,v in enumerate(inputs):
    net.setLowerBound(net.inputVars[0][x], v)
    net.setUpperBound(net.inputVars[0][x], v)
for c in other_cats:
    eq1 = MarabouCore.Equation(MarabouCore.Equation.GE)
    eq1.addAddend(1, net.outputVars[0][c])
    eq1.addAddend(-1, net.outputVars[0][expected_cat])
    eq1.setScalar(0)

eq1 = MarabouCore.Equation(MarabouCore.Equation.LE)
eq1.addAddend(-1, net.outputVars[0][expected_cat])
eq1.addAddend(net.outputVars[])

# y0 -y1 >= 0
# y2 -y1 >= 0
result = net.evaluate([inputs])
print(result)

In [ ]:
# from sensitivity import verify_sensitivity_boundaries
# import pandas as pd

# nnet_path = '../network/models/v3.2.0/model.nnet'
# n_inputs=25
# samples = pd.read_csv('../logs/sensitivity/samples.csv', index_col=0, header=0)
# samples = [(s[0:n_inputs].tolist(), s[n_inputs:].tolist()) for s in samples.values]

# distances = pd.read_csv('../logs/sensitivity/summary.csv', usecols=['x', 'dneg', 'dpos'], index_col='x')
# distances = [tuple(row) for row in distances.values]

# counterexamples = verify_sensitivity_boundaries(nnet_path, samples, distances, precision_adjustment=0, timeout=10, verbose=0)

# print(f'counterexamples in {len(counterexamples.keys())} fields')
# # for ce in counterexamples:
# #     print(ce, counterexamples[ce])

In [ ]:

# from tot_net import TOTNet

# x=0
# s=87
# p=0.00003
# net = TOTNet(nnet_path)
# inputs, outputs = samples[s]
# expected_category = outputs.index(max(outputs))
# other_categories = [o for o in range(len(outputs)) if o != expected_category]
# negd, posd = distances[x][0]+p, distances[x][1]-p
# lbounds=inputs[0:x] + [inputs[x]+negd] + inputs[x+1:]
# ubounds=inputs[0:x] + [inputs[x]+posd] + inputs[x+1:]

# print(f'sample {s}:', samples[s])
# print(f'x{x} dists:', (negd, posd))
# print(f'x{x} lbs:', lbounds[x])
# print(f'x{x} ubs:', ubounds[x])
# res=[]
# for oc in other_categories:
#     net.reset_query()
#     net.set_lower_bounds(lbounds)
#     net.set_upper_bounds(ubounds)
#     net.set_expected_category(oc)
#     vals, _ = net.solve(timeout=10)
#     if len(vals[0]) > 0 or len(vals[1]) > 0:
#         res.append(vals)
#         print('VALS:', vals)

# print('\nRES:', res)